In [2]:
import re
import pandas as pd

In [6]:
dt = pd.read_excel(
    '../data/definitietabel habitattypen (versie 24 maart 2009)_0.xls', 
    sheet_name = 1,
    dtype="string"
)

### TODO: toevoegen codes voor vegetatieloze regels

In [7]:
# Voor de time being:
# Verwijderen van kolommen zonder vegetatiecode. Dit zijn kolommen met waterplantengemeenschappen en vegetatieloze regels.
dt = dt[~dt['Code vegetatietype'].isna()]

### Splitsen vegetatiecode kolom in SBB en VvN

In [8]:
# Opsplitsen van kolom `Code vegetatietype` aparte kolommen voor SBB- en VvN-code
dt['SBB'] = dt['Code vegetatietype']
dt = dt.rename(columns={'Code vegetatietype':'VvN'})
dt['VvN'][dt['VvN'].apply(lambda x: 'SBB' in x)] = None
dt['SBB'][~dt['SBB'].apply(lambda x: 'SBB' in x)] = None

### Opschonen SBB code

In [9]:
# Verwijderen van leading 'SBB-' in SBB-code.
dt['SBB'] = dt['SBB'].str.replace('SBB-', '')

In [17]:
# Er is één SBB code voor klasse SBB-26, daar moet '-xxx [08-f]' afgehaald worden
dt['SBB'] = dt['SBB'].str.replace('-xxx [08-f]', '')

### Opschonen VvN code

In [10]:
def convert_dg_rg_codes(vvn: str):
    '''
    Alleen gebruikt voor de definitietabel habitattypen. Zet codes voor Rompgemeenschappen en Derivaatgemeenschappen om in het format dat gebruikt wordt in de waswordt lijst.
    '''
    try:
        return re.sub('-', '', re.sub('\[.*\]', '', vvn))
    except:
        pass

In [11]:
dt['VvN'] = dt['VvN'].apply(lambda x: convert_dg_rg_codes(x))

### Opgeschoonde tabel opslaan

In [12]:
dt.to_excel('../data/dt_cleaned.xlsx', index=False, header=True)